# Imports

In [29]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [30]:
dataset_name = 'jigsaw unintended bias'
model_name = 'bert'

# change to f'../../results/{dataset_name}' when using inside one of the data subfolders
result_folder = f'../../results/{dataset_name}'
test_csv_filepath = os.path.join(result_folder, 'test.csv')

model_folder = os.path.join(result_folder, model_name) # for this particular model
result_filepath = os.path.join(model_folder, 'results.csv')
dp_result_filepath = os.path.join(model_folder, 'results_dp.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [31]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)

df = pd.read_csv(test_csv_filepath)
# df.drop(columns=['comment_text', 'labels'], inplace=True)
# df.to_csv(test_csv_filepath, index=False)
df.head(2)

,id,male,female,white,black
0,7084460,NaN,NaN,NaN,NaN
1,7141509,NaN,NaN,NaN,NaN


In [32]:
df.fillna(False, inplace=True)

## Split test result

In [33]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

## Merge

In [34]:
result = result.merge(df, on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)

In [35]:
result.head()

,id,labels,probs,male,female,white,black
0,7084460,1,0.712421,False,False,False,False
1,7141509,1,0.885250,False,False,False,False
2,7077814,1,0.731133,False,False,False,False
3,7147990,0,0.289174,False,False,False,False
4,7008066,1,0.866662,False,False,False,False


# Evaluation

## Convert probability to prediction

In [36]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [37]:
group_map = {
    'gender': {
        'unprivileged':['female'],
        'privileged':['male']
    },
    'race': {
        'unprivileged':['black'],
        'privileged': ['white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['female', 'male', 'black', 'white']


### Binarize identity and target columns

In [38]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [39]:
bias_results = {
   # make sure your calculate bias method returns bias metrics in the same order
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.979024,0.999544,0.707993,0.699021
1,Equality of Opportunity (w.r.t y = 1),0.965559,0.962341,0.984192,0.921005
2,Equality of Opportunity (w.r.t y = 0),0.970072,0.951749,0.986702,0.911523
3,Equality of Odds,0.967816,0.957045,0.985447,0.916264
4,unprotected-accuracy,0.741998,0.751503,0.495063,0.614220
5,protected-accuracy,0.724578,0.717054,0.490212,0.568108
6,accuracy,0.733288,0.734279,0.492637,0.591164


In [40]:
bias_results.to_csv(os.path.join(model_folder, 'bias.csv'), index=False)

## Overall metrics

In [41]:
overall_results = {
    'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'false positive rate']
}

for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    privileged_group = subgroup_map['privileged']
    unprivileged_group = subgroup_map['unprivileged']

    privileged_group_name = ','.join(privileged_group)
    unprivileged_group_name = ','.join(unprivileged_group)

    overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
    overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

    overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
    overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

overall_results['Total'] = calculate_metrics(result, [])
overall_results['Total_DP'] = calculate_metrics(dp_result, [])

overall_results = pd.DataFrame(overall_results) 
overall_results

,metrics,male,male_DP,female,female_DP,white,white_DP,black,black_DP,Total,Total_DP
0,auc,0.883173,0.824353,0.880977,0.823837,0.783018,0.733091,0.768420,0.725604,0.940320,0.897022
1,accuracy,0.724578,0.717054,0.741998,0.751503,0.490212,0.568108,0.495063,0.614220,0.865671,0.852272
2,f1_score,0.500000,0.469885,0.474308,0.458351,0.531133,0.554480,0.551724,0.581429,0.506101,0.464504
3,precision,0.345934,0.329144,0.326442,0.323969,0.367983,0.403058,0.390728,0.453735,0.357322,0.326056
4,recall,0.901493,0.820896,0.867052,0.783237,0.954178,0.888140,0.938370,0.809145,0.867167,0.807289
5,true positive rate,0.901493,0.820896,0.867052,0.783237,0.954178,0.888140,0.938370,0.809145,0.867167,0.807289
6,false positive rate,0.307320,0.301668,0.277392,0.253417,0.711111,0.570760,0.724409,0.482283,0.134458,0.143850


In [42]:
overall_results.to_csv(os.path.join(model_folder, 'overall_results.csv'), index=False)